In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from io import BytesIO
import cv2
import bson
from skimage.data import imread
import matplotlib.pyplot as plt
import keras
from PIL import Image
import io, bson, multiprocessing
import os
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

Using TensorFlow backend.


In [2]:
def get_the_data(path):
    data = bson.decode_file_iter(open(path, 'rb'))
    images=[]
    category=[]
    for c, d in enumerate(data):
        product_id = d['_id']
        category_id = d['category_id'] 
        for e, pic in enumerate(d['imgs']):
            category.append(category_id)
            im=pic['picture']
            images.append(Image.open(io.BytesIO(im)))
        if(len(set(category))==860):
            break
    return category, images

In [3]:
product_category_train,image_train=get_the_data('E:\\kaggle\\train.bson')

In [4]:
y, rev_labels = pd.factorize(product_category_train)

In [5]:
len(y)

9998

In [6]:
from sklearn.utils import shuffle
im_train,lab_train=shuffle(image_train,y)
test_im=im_train[9980:]
test_lab=lab_train[9980:]
image_train=im_train[:9980]
label_train=lab_train[:9980]

In [7]:
len_train = len(image_train)
len_test = len(test_im)

In [8]:
len_cat = len(image_train)
#
cnt = 0
flag=0
for i in range(len_cat):
        if i == 0:
            n = label_train[i]
            #print('cat', n)
            dir_ = 'C:\\Users\\saish\\Documents\\EE258\\test\\train'
            pathname = os.path.join(dir_, str(n))
            if not os.path.exists(pathname):
                  os.makedirs(pathname)
            os.chdir(pathname)
            d = image_train[i]
            d.save("{0}.jpg".format(str(cnt)))
        if i != 0:      
            if((label_train[i])!=(label_train[i-1])):
                n = label_train[i]
                #print('cat', n)
                dir_ = 'C:\\Users\\saish\\Documents\\EE258\\test\\train'
                pathname = os.path.join(dir_, str(n))  
                if not os.path.exists(pathname):
                      os.makedirs(pathname)
                os.chdir(pathname)
                d = image_train[i]
                d.save("{0}.jpg".format(str(cnt)))
            if((label_train[i])==(label_train[i-1])):
                  n = label_train[i]
                  #print('cat', n)
                  d = image_train[i]
                  d.save("{0}.jpg".format(str(cnt)))
                
        cnt +=1

In [9]:
len_cat = len(test_im)
#
cnt = 0
flag=0
for i in range(len_cat):
        if i == 0:
            n = test_lab[i]
            #print('cat', n)
            dir_ = 'C:\\Users\\saish\\Documents\\EE258\\test\\test'
            pathname = os.path.join(dir_, str(n))
            if not os.path.exists(pathname):
                  os.makedirs(pathname)
            os.chdir(pathname)
            d = test_im[i]
            d.save("{0}.jpg".format(str(cnt)))
        if i != 0:      
            if((test_lab[i])!=(test_lab[i-1])):
                n = test_lab[i]
                #print('cat', n)
                dir_ = 'C:\\Users\\saish\\Documents\\EE258\\test\\test'
                pathname = os.path.join(dir_, str(n))  
                if not os.path.exists(pathname):
                      os.makedirs(pathname)
                os.chdir(pathname)
                d = test_im[i]
                d.save("{0}.jpg".format(str(cnt)))
            if((test_lab[i])==(test_lab[i-1])):
                  n = test_lab[i]
                  #print('cat', n)
                  d = test_im[i]
                  d.save("{0}.jpg".format(str(cnt)))
                
        cnt +=1

# KERAS ResNet50

In [13]:
img_width, img_height = 197, 197
top_model_weights_path = 'bottleneck_fc_model_Res.h5'
train_data_dir = 'C:\\Users\\saish\\Documents\\EE258\\test\\train'
validation_data_dir = 'C:\\Users\\saish\\Documents\\EE258\\test\\test'
nb_train_samples = len_train
nb_validation_samples = len_test
epochs = 50
batch_size = 1

In [14]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.ResNet50(include_top=False, weights='imagenet',input_shape=(197,197,3))

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train_Res.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation_Res.npy', 'wb'),
            bottleneck_features_validation)

In [15]:
save_bottlebeck_features()

Found 9980 images belonging to 859 classes.
Found 18 images belonging to 10 classes.


In [16]:

train_data = np.load(open('bottleneck_features_train_Res.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('bottleneck_features_validation_Res.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(rev_labels), activation='softmax'))

model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,train_labels,validation_split=0.2,epochs=3,batch_size=1)
    
model.save_weights(top_model_weights_path)


Train on 7984 samples, validate on 1996 samples
Epoch 1/3
7984/7984 [==============================] - 81s 10ms/step - loss: 4.5452 - acc: 0.3312 - val_loss: 4.1907 - val_acc: 0.3307
Epoch 2/3
7984/7984 [==============================] - 75s 9ms/step - loss: 4.6809 - acc: 0.3329 - val_loss: 5.2797 - val_acc: 0.3307
Epoch 3/3
7984/7984 [==============================] - 74s 9ms/step - loss: 5.7335 - acc: 0.3329 - val_loss: 6.2379 - val_acc: 0.3307


In [18]:

train_data = np.load(open('bottleneck_features_train_Res.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('bottleneck_features_validation_Res.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(len(rev_labels), activation='softmax'))

model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,train_labels,validation_split=0.2,epochs=3,batch_size=1)
    
model.save_weights(top_model_weights_path)

Train on 7984 samples, validate on 1996 samples
Epoch 1/3
7984/7984 [==============================] - 217s 27ms/step - loss: 4.4727 - acc: 0.3052 - val_loss: 4.1468 - val_acc: 0.3307
Epoch 2/3
7984/7984 [==============================] - 213s 27ms/step - loss: 4.3144 - acc: 0.3169 - val_loss: 4.5339 - val_acc: 0.3307
Epoch 3/3
7984/7984 [==============================] - 210s 26ms/step - loss: 4.2886 - acc: 0.3235 - val_loss: 4.3657 - val_acc: 0.3307


In [19]:

train_data = np.load(open('bottleneck_features_train_Res.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('bottleneck_features_validation_Res.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(len(rev_labels), activation='softmax'))

model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,train_labels,validation_split=0.2,epochs=3,batch_size=1)
    
#model.save_weights(top_model_weights_path)

Train on 7984 samples, validate on 1996 samples
Epoch 1/3
7984/7984 [==============================] - 394s 49ms/step - loss: 4.4774 - acc: 0.3004 - val_loss: 4.1869 - val_acc: 0.3307
Epoch 2/3
7984/7984 [==============================] - 386s 48ms/step - loss: 4.2818 - acc: 0.3146 - val_loss: 4.6083 - val_acc: 0.3307
Epoch 3/3
7984/7984 [==============================] - 404s 51ms/step - loss: 4.2865 - acc: 0.3203 - val_loss: 4.1987 - val_acc: 0.3307


In [21]:
from keras import regularizers
train_data = np.load(open('bottleneck_features_train_res.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('bottleneck_features_validation_res.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()


model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(len(rev_labels), activation='softmax'))

sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,train_labels,validation_split=0.2,epochs=2,batch_size=1)

Train on 7984 samples, validate on 1996 samples
Epoch 1/2
7984/7984 [==============================] - 291s 37ms/step - loss: 4.4797 - acc: 0.3019 - val_loss: 4.3044 - val_acc: 0.3307
Epoch 2/2
7984/7984 [==============================] - 286s 36ms/step - loss: 4.3409 - acc: 0.3193 - val_loss: 4.1730 - val_acc: 0.3307


In [22]:
from keras import regularizers
train_data = np.load(open('bottleneck_features_train_res.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('bottleneck_features_validation_res.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()


model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(1024, activation='relu',  kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(len(rev_labels), activation='softmax'))

sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,train_labels,validation_split=0.2,epochs=2,batch_size=1)

Train on 7984 samples, validate on 1996 samples
Epoch 1/2
7984/7984 [==============================] - 408s 51ms/step - loss: 8.6019 - acc: 0.3029 - val_loss: 4.6937 - val_acc: 0.3307
Epoch 2/2
7984/7984 [==============================] - 415s 52ms/step - loss: 4.5046 - acc: 0.3189 - val_loss: 4.3982 - val_acc: 0.3307
